In [46]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import Matrix, symbols
from scipy.integrate import nquad
from concurrent.futures import ProcessPoolExecutor
!pip install vegas
import vegas

!pip install pyscf
from pyscf import gto, scf, ao2mo

#1. Define atomic coordinates, nuclear charges and orbitals

In [6]:
## Molecule (e.g. water)

# N.B. distances are in Bohr radii (this will produce a final ground state energy in Hartrees)
# Atomic coordinates of water taken from my UCL CHEM0028 notes

Molecule = {
    "Atom_label": ["O", "H", "H"],
    "Z": [8, 1, 1],
    "x": [0.00000, 0.00000, 0.00000],
    "y": [0.00000, 1.42757, -1.42757],
    "z": [0.00000, -1.11294, -1.11294]
}

df = pd.DataFrame(Molecule)
print(df)


## Orbitals

# Define symbolic coordinates for integration via Sympy

x, y, z = sp.symbols('x y z')
r = (x, y, z)

def sto_3G_1s(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2*alphas[i])/sp.pi)**0.75 * sp.exp(-alphas[i] *((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2s(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2*alphas[i])/sp.pi)**0.75 * sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2px(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*(x - Rx)*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2py(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*(y - Ry)*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2pz(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*((z - Rz))*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

n_electrons = sum(Molecule["Z"])
print("number of electrons =", n_electrons)

basis_set = [sto_3G_1s(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [130.709321, 23.80886605, 6.44360831]),
             sto_3G_2s(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [-0.0999672, 0.39951282, 0.70011546], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2px(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2py(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2pz(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_1s(r, center=(Molecule['x'][1], Molecule['y'][1], Molecule['z'][1]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [3.4252509, 0.62391372, 0.16885540]),
             sto_3G_1s(r, center=(Molecule['x'][2], Molecule['y'][2], Molecule['z'][2]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [3.4252509, 0.62391372, 0.16885540])
]

# N.B. Approximate alpha coefficients taken from https://www.basissetexchange.org/

  Atom_label  Z    x        y        z
0          O  8  0.0  0.00000  0.00000
1          H  1  0.0  1.42757 -1.11294
2          H  1  0.0 -1.42757 -1.11294
number of electrons = 10


#2. Calculating one-electron integrals of the Fock matrix [H_core = T + V_{e-nuc}]

In [11]:
#Constructing T matrix
#precompute laplacians for efficiency

x, y, z, alpha, Rx, Ry, Rz = sp.symbols('x y z alpha Rx Ry Rz')
laplacians = [sp.diff(phi, x, 2) + sp.diff(phi, y, 2) + sp.diff(phi, z, 2) for phi in basis_set]



def t_ij(i, j, basis_set, laplacians):
  integrand = sp.lambdify([[x, y, z]], -0.5 * basis_set[i] * laplacians[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  t = integrator(integrand, nitn=50, neval=10000).mean
  return t


#Parallelisation needed to compute 7x7 matrix within a few mins

def parallel_T_matrix(basis_set, laplacians):
    n = len(basis_set)
    T = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(t_ij, i, j, basis_set, laplacians) for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                T[i, j] = futures[i][j].result()
        # Calculate Oxygen 1s KE separately since numeric integration struggles to capture it
    integrand_sym = -0.5 * basis_set[0] * laplacians[0]  # symbolic expression
    T00_sym = sp.integrate(integrand_sym, (x, -sp.oo, sp.oo), (y, -sp.oo, sp.oo), (z, -sp.oo, sp.oo))
    T[0, 0] = float(T00_sym.evalf())
    return T

T = parallel_T_matrix(basis_set, laplacians)

print(T)

[[ 2.90032031e+01 -1.67743179e-01  1.76324427e-04 -6.43425372e-05
  -1.61104936e-04 -2.55896449e-03 -2.55668523e-03]
 [-1.66899547e-01  8.08558829e-01 -1.81187917e-04 -2.77507627e-04
   6.47749285e-04  1.28779896e-01  1.28731348e-01]
 [-1.79780117e-03  3.65819317e-04  6.32570178e-01 -1.05496494e-04
  -5.57884228e-05 -6.94387541e-05 -1.22030914e-05]
 [-8.18746260e-04 -1.05219395e-04  5.13773658e-04  6.32640665e-01
  -7.89008431e-05  1.12170308e-01 -1.12194957e-01]
 [-1.13041549e-03  1.87738514e-04  1.31879014e-04  7.37572271e-06
   6.32190012e-01 -8.74129600e-02 -8.73650789e-02]
 [-2.37808579e-03  1.32708646e-01  3.34853859e-04  1.13949721e-01
  -8.87781101e-02  7.73936090e-01  1.28824265e-02]
 [-2.51486492e-03  1.32795713e-01 -9.07301548e-05 -1.13900192e-01
  -8.87765660e-02  1.28980152e-02  7.74176859e-01]]


In [15]:
#Constructing V matrix
x, y, z, alpha, Rx, Ry, Rz = sp.symbols('x y z alpha Rx Ry Rz')


#constructing V_func = -Sigma(Z/abs(r - R)) term:
V_funcs = 0
for i in range(len(Molecule["Z"])):
  V_func = - Molecule["Z"][i] / (sp.sqrt((x - Molecule["x"][i])**2 + (y - Molecule["y"][i])**2 + (z - Molecule["z"][i])**2))
  V_funcs += V_func


#precomuting integrands
integrands = [[sp.lambdify((x, y, z), V_funcs * basis_set[i] * basis_set[j], modules='numpy')
              for j in range(len(basis_set))] for i in range(len(basis_set))]

def v_ij(i, j, basis_set):
  integrand = sp.lambdify([[x, y, z]], V_funcs * basis_set[i] * basis_set[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  v = integrator(integrand, nitn=50, neval=10000).mean
  return v

def parallel_V_matrix(basis_set):
    n = len(basis_set)
    V = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(v_ij, i, j, basis_set) for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                V[i, j] = futures[i][j].result()
    return V

V = parallel_V_matrix(basis_set=basis_set)

print(V)


[[-6.171479677537e+01 -7.443531394792e+00 -6.161820499229e-05 -1.100061776993e-03  8.928564862162e-03 -1.744400222760e+00 -1.743566811459e+00]
 [-7.443531394792e+00 -1.014248901800e+01  2.337051949227e-03 -2.905820927958e-03  1.125663496041e-01 -3.859431968836e+00 -3.859286047428e+00]
 [-2.356960423670e-03 -3.874432930116e-04 -2.496189940123e+00 -2.210307038048e-04 -1.588637891736e-04 -4.399993630562e-04 -2.253864854038e-04]
 [ 2.448025054399e-04  3.462131355135e-04 -4.587070018481e-04 -2.533879108311e+00  8.081144164820e-05 -1.119359257739e+00  1.120870991046e+00]
 [ 9.815909248691e-03  1.108006820347e-01  1.227830545261e-03 -7.648306679758e-04 -2.519813190684e+00  9.104171771135e-01  9.103986406897e-01]
 [-1.744130787360e+00 -3.861303510540e+00 -2.525559724291e-04 -1.119152574749e+00  9.104884096866e-01 -5.711412621805e+00 -1.594664385531e+00]
 [-1.743211556914e+00 -3.858909166061e+00  3.065181166956e-04  1.119914994825e+00  9.102875014879e-01 -1.594514284160e+00 -5.708121496405e+00]

#3. Initial guess at the coefficient matrix C using Huckel theory *since bonding has to be specified explicitly, this is only applicable to water


In [141]:
# Obtain AO coefficient eigenvectors, then concatenate them into C
# N.B. we have to ensure that C is a 7x7 matrix so that its dimensionality...
#... matches with the Fock matrix. Therefore we assume that any pair of...
#... orbitals from different atoms = beta.

alpha, beta, E = sp.symbols('alpha beta E')

# Basis: O(1s), O(2s), O(2Px), O(2Py), O(2Pz), H_1(1s), H_2(1s)
# alpha = 0, beta = -1

Secular_Matrix = Matrix([
[- E, 0, 0, 0, 0, -1, -1],
[0, - E, 0, 0, 0, -1, -1],
[0, 0, - E, 0, 0, -1, -1],
[0, 0, 0, - E, 0, -1, -1],
[0, 0, 0, 0, - E, -1, -1],
[-1, -1, -1, -1, -1, - E, 0],
[-1, -1, -1, -1, -1, 0, - E]])

c_eigenvectors = Secular_Matrix.eigenvects()

C = []

for eigval, multiplicity, eigenvec_list in c_eigenvectors:
  for vec in eigenvec_list:
    vec = vec / vec.norm()
    C.append(vec)

C = Matrix.hstack(*C)

sp.pprint(C)

⎡-√2   -√2   -√2   -√2         √10  -√10 ⎤
⎢────  ────  ────  ────   0    ───  ─────⎥
⎢ 2     2     2     2          10    10  ⎥
⎢                                        ⎥
⎢ √2                           √10  -√10 ⎥
⎢ ──    0     0     0     0    ───  ─────⎥
⎢ 2                            10    10  ⎥
⎢                                        ⎥
⎢       √2                     √10  -√10 ⎥
⎢ 0     ──    0     0     0    ───  ─────⎥
⎢       2                      10    10  ⎥
⎢                                        ⎥
⎢             √2               √10  -√10 ⎥
⎢ 0     0     ──    0     0    ───  ─────⎥
⎢             2                10    10  ⎥
⎢                                        ⎥
⎢                   √2         √10  -√10 ⎥
⎢ 0     0     0     ──    0    ───  ─────⎥
⎢                   2          10    10  ⎥
⎢                                        ⎥
⎢                        -√2             ⎥
⎢ 0     0     0     0    ────  1/2   1/2 ⎥
⎢                         2              ⎥
⎢          

#4. Calculating S, U and X such that
#(U^T S U) = s;
#(X^T S X) = I.

In [12]:
# Calculating the overlap integral S
# Write nested function so that it can be reused in the SCF procudure later

def s_ij(i, j, basis_set):
  integrand = sp.lambdify([[x, y, z]], basis_set[i] * basis_set[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  s = integrator(integrand, nitn=50, neval=10000).mean
  return s

# Compute S elements in parallel
def parallel_S_matrix(basis_set):
    n = len(basis_set)
    S = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(s_ij, i, j, basis_set)
                    for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                S[i, j] = futures[i][j].result()
    return S

S = parallel_S_matrix(basis_set)

# Obtaining U and X
eigenvalues, eigenvectors = np.linalg.eig(S)
U = eigenvectors
s_inv_sqrt = np.diag(eigenvalues**-0.5)
X = U @ s_inv_sqrt

S = parallel_S_matrix(basis_set)

sp.pprint(S)

# Check that X^T S X = I
sp.pprint(X.T @ S @ X)


[[ 9.99830790e-01  2.36752259e-01 -3.95700564e-05 -1.95577504e-05
         -1.89595465e-05  5.38922690e-02  5.39008765e-02]        
 [ 2.36752259e-01  9.99701336e-01  9.47409132e-06  3.45653234e-05
          4.75292018e-05  4.72457030e-01  4.72500356e-01]        
 [-3.65538357e-05  1.13344052e-04  2.49887077e-01  1.31789797e-04
         -7.44180026e-05 -2.14853010e-05  9.56823443e-06]        
 [-7.79779529e-05  1.06221794e-04 -1.23848221e-04  2.49908313e-01
          3.58577015e-05  1.53842490e-01 -1.53887887e-01]        
 [ 8.21675818e-07 -2.00569716e-07  6.12221861e-05  5.68001208e-05
          2.49879867e-01 -1.20055628e-01 -1.20058977e-01]        
 [ 5.38928003e-02  4.72465551e-01  4.42117317e-05  1.53926375e-01
         -1.20027864e-01  9.53814006e-01  2.42791079e-01]        
 [ 5.38969687e-02  4.72484206e-01 -2.22074120e-05 -1.53862325e-01
         -1.20055782e-01  2.42776034e-01  9.53541349e-01]]       
[[ 1.00003124e+00 -5.41617147e-05 -2.98449182e-05 -8.19127856e-05
         -

#5. Computing the density matrix P and then coulomb and exchange integrals J and K (takes ~5hrs on Google Colab, see next section to circumvent the J and K calculations)

In [142]:
#Obtaining P = 2 * Simga(occupied MOs)(C_ia^* * C_ja)

n = len(basis_set)
P = np.zeros((n, n))
n_occ = n_electrons // 2

C = np.array(C)
C_occ = C[:, :n_occ]


for i in range(n):
  for mu in range(n):
    for nu in range(n):
      P[mu, nu] += 2*(C[mu, i] * C[nu, i])

P = np.array(P)
sp.pprint(P)

[[ 4.400000000000e+00 -6.000000000000e-01 -6.000000000000e-01 -6.000000000000e ↪
 [-6.000000000000e-01  1.400000000000e+00  4.000000000000e-01  4.000000000000e ↪
 [-6.000000000000e-01  4.000000000000e-01  1.400000000000e+00  4.000000000000e ↪
 [-6.000000000000e-01  4.000000000000e-01  4.000000000000e-01  1.400000000000e ↪
 [-6.000000000000e-01  4.000000000000e-01  4.000000000000e-01  4.000000000000e ↪
 [ 7.976586724465e-18  7.976586724465e-18  7.976586724465e-18  7.976586724465e ↪
 [ 7.976586724465e-18  7.976586724465e-18  7.976586724465e-18  7.976586724465e ↪

↪ -01 -6.000000000000e-01  7.976586724465e-18  7.976586724465e-18] 
↪ -01  4.000000000000e-01  7.976586724465e-18  7.976586724465e-18] 
↪ -01  4.000000000000e-01  7.976586724465e-18  7.976586724465e-18] 
↪ +00  4.000000000000e-01  7.976586724465e-18  7.976586724465e-18] 
↪ -01  1.400000000000e+00  7.976586724465e-18  7.976586724465e-18] 
↪ -18  7.976586724465e-18  2.000000000000e+00  0.000000000000e+00] 
↪ -18  7.976586724465e-1

In [ ]:
x1, y1, z1, x2, y2, z2 = sp.symbols('x1 y1 z1 x2 y2 z2')

# J = SIGMA(lambda sigma)(P_{lambda sigma} * (mu nu | lambda sigma))
# First define the 4-index integral (mu nu | lambda sigma):

def r12_inv(x1, y1, z1, x2, y2, z2):
    return (1 / np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2))

numeric_J_integrand = np.empty((len(basis_set),)*4, dtype=object)
numeric_K_integrand = np.empty((len(basis_set),)*4, dtype=object)

def integrand_JK(x1, y1, z1, x2, y2, z2, bf1, bf2, bf3, bf4):
    r12 = sp.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
    return bf1.subs({x: x1, y: y1, z: z1}) * bf2.subs({x: x1, y: y1, z: z1}) * (1 / r12) * bf3.subs({x: x2, y: y2, z: z2}) * bf4.subs({x: x2, y: y2, z: z2})

def integrand_JK_symbolic(bf1, bf2, bf3, bf4):
    return bf1 * bf2 * (1 / sp.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)) * bf3 * bf4

numeric_J_integrand = np.empty((len(basis_set),)*4, dtype=object)
numeric_K_integrand = np.empty((len(basis_set),)*4, dtype=object)

for i in range(len(basis_set)):
    for j in range(len(basis_set)):
        for k in range(len(basis_set)):
            for l in range(len(basis_set)):
                symbolic_J_integrand = integrand_JK_symbolic(
                    basis_set[i].subs({x: x1, y: y1, z: z1}),
                    basis_set[j].subs({x: x1, y: y1, z: z1}),
                    basis_set[k].subs({x: x2, y: y2, z: z2}),
                    basis_set[l].subs({x: x2, y: y2, z: z2}),
                )
                numeric_J_integrand[i,j,k,l] = sp.lambdify(((x1, y1, z1, x2, y2, z2),), symbolic_J_integrand, 'numpy')


                symbolic_K_integrand = integrand_JK_symbolic(
                    basis_set[i].subs({x: x1, y: y1, z: z1}),
                    basis_set[k].subs({x: x1, y: y1, z: z1}),
                    basis_set[j].subs({x: x2, y: y2, z: z2}),
                    basis_set[l].subs({x: x2, y: y2, z: z2}),
                )
                numeric_K_integrand[i,j,k,l] = sp.lambdify(((x1, y1, z1, x2, y2, z2),), symbolic_K_integrand, 'numpy')


# Use vegas monte carlo integrator (faster than analytic integrators)
# Re-initialise the integrator on each run so that its monte carlo adaptive sampling doesnt carry over



def j_ij(i, j):
  j_term = 0
  integrator = vegas.Integrator([[-3,3], [-3,3], [-3,3], [-3,3], [-3,3], [-3,3]])
  for k in range(len(basis_set)):
    for l in range(len(basis_set)):
      j_term += P[k][l] * integrator(numeric_J_integrand[i,j,k,l], nitn=10, neval=10000).mean
  return j_term


def k_ij(i, j):
  k_term = 0
  integrator = vegas.Integrator([[-3,3], [-3,3], [-3,3], [-3,3], [-3,3], [-3,3]])
  for k in range(len(basis_set)):
    for l in range(len(basis_set)):
      k_term += P[k][l] * integrator(numeric_K_integrand[i,k,j,l], nitn=10, neval=10000).mean
  return k_term


# P is just a coefficient matrix so we can multiply it by the integrands pre-integration.

def parallel_J_matrix(basis_set):
    n = len(basis_set)
    J = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(j_ij, i, j) for i in range(n)] for j in range(n)]
        for i in range(n):
            for j in range(n):
                J[i, j] = futures[i][j].result()
    return J

def parallel_K_matrix(basis_set):
    n = len(basis_set)
    K = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(k_ij, i, j) for i in range(n)] for j in range(n)]
        for i in range(n):
            for j in range(n):
                K[i, j] = futures[i][j].result()
    return K

J = parallel_J_matrix(basis_set)
K = parallel_K_matrix(basis_set)

sp.pprint(J)
sp.pprint(K)

F = np.zeros((n, n))
F[i, j] = T[i, j] + V[i, j] + J[i, j] - 0.5 * K[i, j]

sp.pprint(F)

# See below: Electron Repulsion Integral matrix (7x7x7x7 matrix) calculated using an external computer (RunPod H100 PCLe).

In [135]:
eri = [[[[2.110646068652057e-06, 3.8116694954486175e-05, 7.374103539502885e-06, -0.002025561220844455, -0.001347373877710136, 0.022267525074023773, 4.851490824336389e-05], [3.8116694954486175e-05, 1.1098820282238009, 0.003210109518748592, -0.002372272239591545, 0.0010572226409878364, 0.3644845439441328, 0.4029620579613538], [7.374103539502885e-06, 0.003210109518748592, 0.190899015677332, -0.0003977517931214619, 0.001667507829130732, 0.0005780326601597414, 0.0006112624549165606], [-0.002025561220844455, -0.002372272239591545, -0.0003977517931214619, 0.27722330887011504, 6.121720965474722e-05, 0.11043082133101192, -0.11206135001811636], [-0.001347373877710136, 0.0010572226409878364, 0.001667507829130732, 6.121720965474722e-05, 0.2473963605333805, -0.08419827364404044, -0.08918171192738722], [0.022267525074023773, 0.3644845439441328, 0.0005780326601597414, 0.11043082133101192, -0.08419827364404044, 0.5149328656525697, 0.15792511117601882], [4.851490824336389e-05, 0.4029620579613538, 0.0006112624549165606, -0.11206135001811636, -0.08918171192738722, 0.15792511117601882, 0.5170677309939025]], [[3.8116694954486175e-05, 0.016580144985925655, 2.1918022821509916e-05, -0.0003268139030133377, 0.0001183361433619371, 0.02259288695322584, 0.03135261126841168], [0.016580144985925655, 0.25479668985854703, 0.0002980075972139225, 5.8345257364352e-05, 0.0001770103305669025, 0.09352806207030455, 0.09328429938955632], [2.1918022821509916e-05, 0.0002980075972139225, 0.06437229959561983,-0.0003356177562167984, 0.00020946196158225222, 0.00019677063885580035, -1.8227093133976622e-05], [-0.0003268139030133377, 5.8345257364352e-05, -0.0003356177562167984, 0.06444595587560598, 0.00013188501831550432, 0.026400370652775174, -0.02620931048790621], [0.0001183361433619371, 0.0001770103305669025, 0.00020946196158225222, 0.00013188501831550432, 0.06391907629302358, -0.02038632354573043, -0.02040994400670298], [0.02259288695322584, 0.09352806207030455, 0.00019677063885580035, 0.026400370652775174, -0.02038632354573043,0.1225656953736262, 0.03734740580653984], [0.03135261126841168, 0.09328429938955632, -1.8227093133976622e-05, -0.02620931048790621, -0.02040994400670298, 0.03734740580653984, 0.12283131547133579]], [[7.374103539502885e-06, 2.1918022821509916e-05, 0.005920438316907536, 8.184429333199754e-06, 4.5832447490243114e-05, 3.382843917673257e-06, -2.1658884366044888e-05], [2.1918022821509916e-05, -0.0006551815306482724, 0.009192991926412097, 0.00022595427064614636, 0.00024644502726346463, 9.173333466272427e-05, 0.00010535142003172982], [0.005920438316907536, 0.009192991926412097, -0.00017787217294952488, -6.520339283819167e-05, 6.81974160157242e-05, 0.0025886606580982218, 0.0024911255398025658], [8.184429333199754e-06, 0.00022595427064614636, -6.520339283819167e-05, -5.122476762497382e-05, -0.0001072088189127408, -2.508723781309922e-05, 8.21415173114361e-05], [4.5832447490243114e-05, 0.00024644502726346463, 6.81974160157242e-05, -0.0001072088189127408, -8.501073149346723e-05, -0.00010298315084115836, 9.348261321963548e-05], [3.382843917673257e-06, 9.173333466272427e-05, 0.0025886606580982218, -2.508723781309922e-05, -0.00010298315084115836, -0.00029931530084244463, 2.418409961740715e-06], [-2.1658884366044888e-05, 0.00010535142003172982, 0.0024911255398025658, 8.21415173114361e-05, 9.348261321963548e-05, 2.418409961740715e-06, 2.8845205573384447e-06]], [[-0.002025561220844455, -0.0003268139030133377, 8.184429333199754e-06, 0.0002693295381250127, -0.0001123273381929337,0.0005494868156957986, -1.5310508664913407e-05], [-0.0003268139030133377, -0.000430085186594592, 0.000681312497410403, 0.009132595396969144, -3.0355259318597374e-05, 0.003338179499542315, -0.002731552038356108],[8.184429333199754e-06, 0.000681312497410403, -1.8621107010644324e-05, -0.00021615801381406706, 3.0189426661872054e-05, -3.973656192514144e-06, 2.4377154310510185e-05], [0.0002693295381250127, 0.009132595396969144, -0.00021615801381406706, -0.0001032203144607894, -7.444270112187904e-05, 0.0030109871948317563, 0.003123884493153085], [-0.0001123273381929337, -3.0355259318597374e-05, 3.0189426661872054e-05, -7.444270112187904e-05, 4.127458831340487e-05, -0.0005976797023395749, 0.0004825706122380642], [0.0005494868156957986, 0.003338179499542315, -3.973656192514144e-06, 0.0030109871948317563, -0.0005976797023395749, 0.0047679417827660945, 7.258810714984401e-05], [-1.5310508664913407e-05, -0.002731552038356108, 2.4377154310510185e-05, 0.003123884493153085, 0.0004825706122380642, 7.258810714984401e-05, -0.004884236643421517]], [[-0.001347373877710136, 0.0001183361433619371, 4.5832447490243114e-05, -0.0001123273381929337, 0.006127301002066336, -0.00040745246894667526, -0.0003072384807615293], [0.0001183361433619371, -0.0007766239108986438, -0.00014537054807644043, -0.00013960022773271442, 0.008828512135188161, -0.0023116684027265385, -0.0020724902449363444], [4.5832447490243114e-05, -0.00014537054807644043, 1.8116830789427017e-05, -8.993674430477472e-05, 4.296156264657606e-05, 0.00013366843495048208, -5.375790698520388e-05], [-0.0001123273381929337, -0.00013960022773271442, -8.993674430477472e-05, 6.380603181798042e-06, 4.056957403487189e-05, -0.0005296233681836721,0.0005683521578960179], [0.006127301002066336, 0.008828512135188161, 4.296156264657606e-05, 4.056957403487189e-05, 5.406889972942455e-05, 0.003003656398226684, 0.0029535866730781273], [-0.00040745246894667526, -0.0023116684027265385, 0.00013366843495048208, -0.0005296233681836721, 0.003003656398226684, -0.0037830812938639533, -0.0012022442002726254], [-0.0003072384807615293, -0.0020724902449363444, -5.375790698520388e-05, 0.0005683521578960179, 0.0029535866730781273, -0.0012022442002726254, -0.0038752861527438657]], [[0.022267525074023773, 0.02259288695322584, 3.382843917673257e-06, 0.0005494868156957986, -0.00040745246894667526, 0.00727584391403289, 0.007096490665074045], [0.02259288695322584, 0.05855032980910715, -5.204095066859523e-05, 0.0010791432403306873, -0.0013324615081283947, 0.021706485700218457, 0.021132401282331598], [3.382843917673257e-06, -5.204095066859523e-05, 0.014696447039377591, -7.493449219567007e-05, -6.900536157589576e-05, -1.949663972696094e-05, -5.83972063184236e-05], [0.0005494868156957986, 0.0010791432403306873, -7.493449219567007e-05, 0.014596834735635524, -1.648645910972081e-05, 0.006309958390147072, -0.005591446303681881], [-0.00040745246894667526, -0.0013324615081283947, -6.900536157589576e-05, -1.648645910972081e-05, 0.014734843186487162, -0.004958622289343094, -0.004813244128911958], [0.00727584391403289, 0.021706485700218457, -1.949663972696094e-05, 0.006309958390147072, -0.004958622289343094, 0.028837548465787385, 0.008578369344337587], [0.007096490665074045, 0.021132401282331598, -5.83972063184236e-05, -0.005591446303681881, -0.004813244128911958, 0.008578369344337587, 0.02767979868837847]], [[4.851490824336389e-05, 0.03135261126841168, -2.1658884366044888e-05, -1.5310508664913407e-05, -0.0003072384807615293, 0.007096490665074045, 0.007146804536940045], [0.03135261126841168, 0.05771947798490009, -6.474711884021497e-05, -0.001029428906515696, -0.0009210566467858411, 0.021266390375878876, 0.02184813359392102], [-2.1658884366044888e-05, -6.474711884021497e-05, 0.014563452548164666, 6.987536770642768e-05, 3.965441572165417e-05, 3.540002224385605e-05, 7.883345806673966e-05], [-1.5310508664913407e-05, -0.001029428906515696, 6.987536770642768e-05, 0.014563977287950938, 3.198903933865263e-05, 0.005630234307734144, -0.006395962020397226], [-0.0003072384807615293, -0.0009210566467858411, 3.965441572165417e-05, 3.198903933865263e-05, 0.01462532540516598, -0.004900035256751265, -0.004917403296289849], [0.007096490665074045, 0.021266390375878876, 3.540002224385605e-05, 0.005630234307734144, -0.004900035256751265, 0.02770084619198524, 0.00857444586700688], [0.007146804536940045, 0.02184813359392102, 7.883345806673966e-05, -0.006395962020397226, -0.004917403296289849, 0.00857444586700688, 0.028924623004966243]]], [[[3.8116694954486175e-05, 0.016580144985925655, 2.1918022821509916e-05, -0.0003268139030133377, 0.0001183361433619371, 0.02259288695322584, 0.03135261126841168], [0.016580144985925655, 0.25479668985854703, 0.0002980075972139225, 5.8345257364352e-05, 0.0001770103305669025, 0.09352806207030455, 0.09328429938955632], [2.1918022821509916e-05, 0.0002980075972139225, 0.06437229959561983, -0.0003356177562167984, 0.00020946196158225222, 0.00019677063885580035, -1.8227093133976622e-05], [-0.0003268139030133377, 5.8345257364352e-05, -0.0003356177562167984, 0.06444595587560598, 0.00013188501831550432, 0.026400370652775174, -0.02620931048790621], [0.0001183361433619371, 0.0001770103305669025, 0.00020946196158225222,0.00013188501831550432, 0.06391907629302358, -0.02038632354573043, -0.02040994400670298], [0.02259288695322584, 0.09352806207030455, 0.00019677063885580035, 0.026400370652775174, -0.02038632354573043, 0.1225656953736262, 0.03734740580653984], [0.03135261126841168, 0.09328429938955632, -1.8227093133976622e-05, -0.02620931048790621, -0.02040994400670298, 0.03734740580653984, 0.12283131547133579]], [[1.1098820282238009, 0.25479668985854703, -0.0006551815306482724, -0.000430085186594592, -0.0007766239108986438, 0.05855032980910715, 0.05771947798490009], [0.25479668985854703, 0.8208036828764155, 0.00045998810262718506, 0.0006616595974712478, 0.0018136637534886739, 0.3287857585908965, 0.32698320176677725], [-0.0006551815306482724, 0.00045998810262718506, 0.20384429734856688, 7.527074214120831e-05, -0.0007768429809752506, 0.00048227219482233183, -0.0002905224191685269], [-0.000430085186594592, 0.0006616595974712478, 7.527074214120831e-05, 0.20512243441159506, 0.0003435370124715998, 0.09909683634178008, -0.09827396222592277], [-0.0007766239108986438, 0.0018136637534886739, -0.0007768429809752506, 0.0003435370124715998, 0.2042737745292473, -0.07634388612024146, -0.07729915678535447], [0.05855032980910715, 0.3287857585908965, 0.00048227219482233183, 0.09909683634178008, -0.07634388612024146, 0.4934603396670194, 0.14093896484286952], [0.05771947798490009, 0.32698320176677725, -0.0002905224191685269, -0.09827396222592277, -0.07729915678535447, 0.14093896484286952, 0.49648660188975735]], [[0.003210109518748592, 0.0002980075972139225, 0.009192991926412097, 0.000681312497410403, -0.00014537054807644043, -5.204095066859523e-05, -6.474711884021497e-05], [0.0002980075972139225, 0.00045998810262718506, 0.04458382322006207, 0.0002483529766465245, 0.0003892658738344131, -0.0007571370722339967, -0.0006340975237355889], [0.009192991926412097, 0.04458382322006207, -5.459398717077135e-05, -2.07504197917624e-05, -8.26602344348717e-05, 0.014428810501622563, 0.014867452577030362], [0.000681312497410403, 0.0002483529766465245, -2.07504197917624e-05, -0.0004775908086346015, -8.646390311496467e-07, -0.0008259471732854484, 2.4501508025723094e-05], [-0.00014537054807644043, 0.0003892658738344131, -8.26602344348717e-05,-8.646390311496467e-07, 0.0005825329574675204, 0.0006080145024423654, 0.0004348997609271964], [-5.204095066859523e-05, -0.0007571370722339967, 0.014428810501622563, -0.0008259471732854484, 0.0006080145024423654,0.001174381271699723, -0.00022825956995994735], [-6.474711884021497e-05, -0.0006340975237355889, 0.014867452577030362, 2.4501508025723094e-05, 0.0004348997609271964, -0.00022825956995994735, 0.0007511070389618449]], [[-0.002372272239591545, 5.8345257364352e-05, 0.00022595427064614636, 0.009132595396969144, -0.00013960022773271442, 0.0010791432403306873, -0.001029428906515696], [5.8345257364352e-05, 0.0006616595974712478, 0.0002483529766465245, 0.04524699964717533, 0.0005541274947928986, 0.021340803621331568, -0.019793430266554446], [0.00022595427064614636, 0.0002483529766465245, -0.00014043409890485896, 0.0007423176329667283, 7.643487654402129e-05, 0.00020524942867038474, -0.0002672015710464327], [0.009132595396969144, 0.04524699964717533, 0.0007423176329667283, -0.0008972487086437229, 4.210629030963312e-05, 0.01994373582236024, 0.020667085763733425], [-0.00013960022773271442, 0.0005541274947928986, 7.643487654402129e-05, 4.210629030963312e-05, 0.00017645064888124678, -0.004590887129203333, 0.005255603973000529], [0.0010791432403306873, 0.021340803621331568, 0.00020524942867038474, 0.01994373582236024, -0.004590887129203333, 0.048251761048335666, 0.0005233161170690087], [-0.001029428906515696, -0.019793430266554446, -0.0002672015710464327, 0.020667085763733425, 0.005255603973000529, 0.0005233161170690087, -0.05008641718898815]], [[0.0010572226409878364, 0.0001770103305669025, 0.00024644502726346463, -3.0355259318597374e-05, 0.008828512135188161, -0.0013324615081283947, -0.0009210566467858411], [0.0001770103305669025, 0.0018136637534886739, 0.0003892658738344131, 0.0005541274947928986, 0.045920550119929245, -0.015406296197530443, -0.015883485844307852], [0.00024644502726346463, 0.0003892658738344131, -0.0004113090336712422, -7.958484674011123e-05, 7.355501211793633e-05, -0.00034317364650605133, -0.00018341887503059632], [-3.0355259318597374e-05, 0.0005541274947928986, -7.958484674011123e-05, 0.000823714107255858, -0.00014870634256568182, -0.0040746475288375335, 0.004839335671706909], [0.008828512135188161, 0.045920550119929245, 7.355501211793633e-05, -0.00014870634256568182, 8.03775988054784e-05, 0.018139700242145614, 0.01765235222968264], [-0.0013324615081283947, -0.015406296197530443, -0.00034317364650605133, -0.0040746475288375335, 0.018139700242145614, -0.03879428840396135, -0.012397286852518024], [-0.0009210566467858411, -0.015883485844307852, -0.00018341887503059632, 0.004839335671706909, 0.01765235222968264, -0.012397286852518024, -0.03865119997661427]], [[0.3644845439441328, 0.09352806207030455, 9.173333466272427e-05, 0.003338179499542315, -0.0023116684027265385, 0.021706485700218457, 0.021266390375878876], [0.09352806207030455, 0.3287857585908965, -0.0007571370722339967, 0.021340803621331568, -0.015406296197530443, 0.1606031146011193, 0.13181022936342057], [9.173333466272427e-05, -0.0007571370722339967, 0.08081221640225682, 0.00014384971915712617, -4.757918058930414e-06, 0.00022194626511465074, -0.0002517198022295159], [0.003338179499542315, 0.021340803621331568, 0.00014384971915712617, 0.08269174984849849, -0.001846690953169533, 0.05881380997639974, -0.033123083535107324], [-0.0023116684027265385, -0.015406296197530443, -4.757918058930414e-06, -0.001846690953169533, 0.08359643466535029, -0.045696239623959196, -0.036892983182761255], [0.021706485700218457, 0.1606031146011193, 0.00022194626511465074, 0.05881380997639974, -0.045696239623959196, 0.29687599719188684, 0.06847117548415625], [0.021266390375878876, 0.13181022936342057, -0.0002517198022295159, -0.033123083535107324, -0.036892983182761255, 0.06847117548415625, 0.20165099012913748]], [[0.4029620579613538, 0.09328429938955632, 0.00010535142003172982, -0.002731552038356108, -0.0020724902449363444, 0.021132401282331598, 0.02184813359392102], [0.09328429938955632, 0.32698320176677725, -0.0006340975237355889, -0.019793430266554446, -0.015883485844307852, 0.13181022936342057, 0.15941703232142165], [0.00010535142003172982, -0.0006340975237355889, 0.08006187313655948, -1.2905000916901254e-05, -9.314844761874187e-06, 0.0004362248572625858, 0.00019962718355921692], [-0.002731552038356108, -0.019793430266554446, -1.2905000916901254e-05, 0.08285864175103555, 0.00197583680460194, 0.03253185741678225, -0.05886117450806254], [-0.0020724902449363444, -0.015883485844307852, -9.314844761874187e-06, 0.00197583680460194,0.0813627811635329, -0.036367055110120235, -0.045731910481121296], [0.021132401282331598, 0.13181022936342057, 0.0004362248572625858, 0.03253185741678225, -0.036367055110120235, 0.2023122645187754, 0.06813959308195257], [0.02184813359392102, 0.15941703232142165, 0.00019962718355921692, -0.05886117450806254, -0.045731910481121296, 0.06813959308195257, 0.2993652344262104]]], [[[7.374103539502885e-06, 2.1918022821509916e-05, 0.005920438316907536, 8.184429333199754e-06, 4.5832447490243114e-05, 3.382843917673257e-06, -2.1658884366044888e-05], [2.1918022821509916e-05, -0.0006551815306482724, 0.009192991926412097, 0.00022595427064614636, 0.00024644502726346463, 9.173333466272427e-05, 0.00010535142003172982], [0.005920438316907536, 0.009192991926412097, -0.00017787217294952488, -6.520339283819167e-05, 6.81974160157242e-05, 0.0025886606580982218, 0.0024911255398025658], [8.184429333199754e-06, 0.00022595427064614636, -6.520339283819167e-05, -5.122476762497382e-05, -0.0001072088189127408, -2.508723781309922e-05, 8.21415173114361e-05], [4.5832447490243114e-05, 0.00024644502726346463, 6.81974160157242e-05, -0.0001072088189127408, -8.501073149346723e-05, -0.00010298315084115836, 9.348261321963548e-05], [3.382843917673257e-06, 9.173333466272427e-05, 0.0025886606580982218, -2.508723781309922e-05, -0.00010298315084115836, -0.00029931530084244463, 2.418409961740715e-06], [-2.1658884366044888e-05, 0.00010535142003172982, 0.0024911255398025658, 8.21415173114361e-05, 9.348261321963548e-05, 2.418409961740715e-06, 2.8845205573384447e-06]], [[0.003210109518748592, 0.0002980075972139225, 0.009192991926412097, 0.000681312497410403, -0.00014537054807644043, -5.204095066859523e-05, -6.474711884021497e-05], [0.0002980075972139225, 0.00045998810262718506, 0.04458382322006207, 0.0002483529766465245, 0.0003892658738344131, -0.0007571370722339967, -0.0006340975237355889], [0.009192991926412097, 0.04458382322006207, -5.459398717077135e-05, -2.07504197917624e-05, -8.26602344348717e-05, 0.014428810501622563, 0.014867452577030362], [0.000681312497410403, 0.0002483529766465245, -2.07504197917624e-05, -0.0004775908086346015, -8.646390311496467e-07, -0.0008259471732854484, 2.4501508025723094e-05], [-0.00014537054807644043, 0.0003892658738344131, -8.26602344348717e-05, -8.646390311496467e-07, 0.0005825329574675204, 0.0006080145024423654, 0.0004348997609271964], [-5.204095066859523e-05, -0.0007571370722339967, 0.014428810501622563, -0.0008259471732854484, 0.0006080145024423654, 0.001174381271699723, -0.00022825956995994735], [-6.474711884021497e-05, -0.0006340975237355889, 0.014867452577030362, 2.4501508025723094e-05, 0.0004348997609271964, -0.00022825956995994735, 0.0007511070389618449]], [[0.190899015677332, 0.06437229959561983, -0.00017787217294952488, -1.8621107010644324e-05, 1.8116830789427017e-05, 0.014696447039377591, 0.014563452548164666], [0.06437229959561983, 0.20384429734856688, -5.459398717077135e-05, -0.00014043409890485896, -0.0004113090336712422, 0.08081221640225682, 0.08006187313655948], [-0.00017787217294952488, -5.459398717077135e-05, 0.05510680008336052, 0.00029893179122863757, 0.00020712531414140026, 5.4115183449543824e-05, 0.00014710386548829533], [-1.8621107010644324e-05, -0.00014043409890485896, 0.00029893179122863757, 0.048850320895663844, -1.461217285854909e-05, 0.02350466222068911, -0.023457903750337957], [1.8116830789427017e-05, -0.0004113090336712422, 0.00020712531414140026, -1.461217285854909e-05, 0.04945291619073747, -0.018128283665638477, -0.0181297242858708], [0.014696447039377591, 0.08081221640225682, 5.4115183449543824e-05, 0.02350466222068911,-0.018128283665638477, 0.117820832664402, 0.03445784658758866], [0.014563452548164666, 0.08006187313655948, 0.00014710386548829533, -0.023457903750337957, -0.0181297242858708, 0.03445784658758866, 0.11742614051785429]], [[-0.0003977517931214619, -0.0003356177562167984, -6.520339283819167e-05, -0.00021615801381406706, -8.993674430477472e-05, -7.493449219567007e-05, 6.987536770642768e-05], [-0.0003356177562167984, 7.527074214120831e-05, -2.07504197917624e-05, 0.0007423176329667283, -7.958484674011123e-05, 0.00014384971915712617, -1.2905000916901254e-05], [-6.520339283819167e-05, -2.07504197917624e-05, 0.00029893179122863757, 0.00305980776567675, -8.472785031978486e-05, 0.0011547127645856184, -0.001121133155181657], [-0.00021615801381406706, 0.0007423176329667283, 0.00305980776567675, 8.059343704781186e-05, 0.0001374402046325693, 2.52904946907122e-05, -0.00013246281919940286], [-8.993674430477472e-05, -7.958484674011123e-05, -8.472785031978486e-05, 0.0001374402046325693, -0.00023379547601553688, -0.0001372217195282382, 0.0002555986987095231], [-7.493449219567007e-05, 0.00014384971915712617, 0.0011547127645856184, 2.52904946907122e-05, -0.0001372217195282382, 5.347078666406683e-06, 5.604521858145438e-06], [6.987536770642768e-05, -1.2905000916901254e-05, -0.001121133155181657, -0.00013246281919940286, 0.0002555986987095231, 5.604521858145438e-06, 0.00032066385133268106]], [[0.001667507829130732, 0.00020946196158225222, 6.81974160157242e-05, 3.0189426661872054e-05,4.296156264657606e-05, -6.900536157589576e-05, 3.965441572165417e-05], [0.00020946196158225222, -0.0007768429809752506, -8.26602344348717e-05, 7.643487654402129e-05, 7.355501211793633e-05, -4.757918058930414e-06, -9.314844761874187e-06], [6.81974160157242e-05, -8.26602344348717e-05, 0.00020712531414140026, -8.472785031978486e-05, 0.0029552031455634975, -0.0008517243170561821, -0.0009334987190590691], [3.0189426661872054e-05, 7.643487654402129e-05, -8.472785031978486e-05, -0.00042653821940523987, -0.00015783216485643742, 0.00014527905232747124, 0.00022287427477036244], [4.296156264657606e-05, 7.355501211793633e-05, 0.0029552031455634975, -0.00015783216485643742, 0.00024009755127169362, -0.0003107056100034168, -0.00016210117363589283], [-6.900536157589576e-05, -4.757918058930414e-06, -0.0008517243170561821, 0.00014527905232747124, -0.0003107056100034168, -0.00013540768263552521, -0.00012291048196992792], [3.965441572165417e-05, -9.314844761874187e-06, -0.0009334987190590691, 0.00022287427477036244, -0.00016210117363589283, -0.00012291048196992792, -0.00038059268003511786]], [[0.0005780326601597414, 0.00019677063885580035, 0.0025886606580982218, -3.973656192514144e-06, 0.00013366843495048208, -1.949663972696094e-05, 3.540002224385605e-05], [0.00019677063885580035, 0.00048227219482233183, 0.014428810501622563, 0.00020524942867038474, -0.00034317364650605133, 0.00022194626511465074, 0.0004362248572625858], [0.0025886606580982218, 0.014428810501622563, 5.4115183449543824e-05, 0.0011547127645856184, -0.0008517243170561821, 0.0055159724439013316, 0.004621082030466618], [-3.973656192514144e-06, 0.00020524942867038474, 0.0011547127645856184, -9.229120105623288e-06, 0.0002428108883273125, 0.0003264507807248733, 1.821817150769628e-05], [0.00013366843495048208, -0.00034317364650605133, -0.0008517243170561821, 0.0002428108883273125, -0.0002664188587632032, 0.00016137775509847144, 0.00025065663580608934], [-1.949663972696094e-05, 0.00022194626511465074, 0.0055159724439013316, 0.0003264507807248733, 0.00016137775509847144, 0.0002311917370747674, -6.853916750750541e-05], [3.540002224385605e-05, 0.0004362248572625858, 0.004621082030466618, 1.821817150769628e-05, 0.00025065663580608934, -6.853916750750541e-05, 0.0003106496179318075]], [[0.0006112624549165606, -1.8227093133976622e-05, 0.0024911255398025658, 2.4377154310510185e-05, -5.375790698520388e-05, -5.83972063184236e-05, 7.883345806673966e-05], [-1.8227093133976622e-05, -0.0002905224191685269, 0.014867452577030362, -0.0002672015710464327, -0.00018341887503059632, -0.0002517198022295159, 0.00019962718355921692], [0.0024911255398025658, 0.014867452577030362, 0.00014710386548829533, -0.001121133155181657, -0.0009334987190590691, 0.004621082030466618, 0.005702058346928917],[2.4377154310510185e-05, -0.0002672015710464327, -0.001121133155181657, 0.00027292654501881355, 0.00021019560190190965, 2.976024214903177e-05, -0.00024251519800809452], [-5.375790698520388e-05, -0.00018341887503059632, -0.0009334987190590691, 0.00021019560190190965, 6.163143713962752e-05, 5.580975283204249e-05, 4.959757028833822e-05], [-5.83972063184236e-05, -0.0002517198022295159, 0.004621082030466618, 2.976024214903177e-05, 5.580975283204249e-05, -0.0005649198133347628, 2.168793935338556e-05], [7.883345806673966e-05, 0.00019962718355921692, 0.005702058346928917, -0.00024251519800809452, 4.959757028833822e-05, 2.168793935338556e-05, 0.00046133282878836726]]], [[[-0.002025561220844455, -0.0003268139030133377, 8.184429333199754e-06,0.0002693295381250127, -0.0001123273381929337, 0.0005494868156957986, -1.5310508664913407e-05], [-0.0003268139030133377, -0.000430085186594592, 0.000681312497410403, 0.009132595396969144, -3.0355259318597374e-05, 0.003338179499542315, -0.002731552038356108], [8.184429333199754e-06, 0.000681312497410403, -1.8621107010644324e-05, -0.00021615801381406706, 3.0189426661872054e-05, -3.973656192514144e-06, 2.4377154310510185e-05], [0.0002693295381250127, 0.009132595396969144, -0.00021615801381406706, -0.0001032203144607894, -7.444270112187904e-05, 0.0030109871948317563, 0.003123884493153085], [-0.0001123273381929337, -3.0355259318597374e-05, 3.0189426661872054e-05, -7.444270112187904e-05, 4.127458831340487e-05, -0.0005976797023395749, 0.0004825706122380642], [0.0005494868156957986, 0.003338179499542315, -3.973656192514144e-06, 0.0030109871948317563, -0.0005976797023395749, 0.0047679417827660945, 7.258810714984401e-05], [-1.5310508664913407e-05, -0.002731552038356108, 2.4377154310510185e-05, 0.003123884493153085, 0.0004825706122380642, 7.258810714984401e-05, -0.004884236643421517]], [[-0.002372272239591545, 5.8345257364352e-05, 0.00022595427064614636, 0.009132595396969144, -0.00013960022773271442, 0.0010791432403306873, -0.001029428906515696], [5.8345257364352e-05, 0.0006616595974712478, 0.0002483529766465245, 0.04524699964717533, 0.0005541274947928986, 0.021340803621331568, -0.019793430266554446], [0.00022595427064614636, 0.0002483529766465245, -0.00014043409890485896, 0.0007423176329667283, 7.643487654402129e-05, 0.00020524942867038474, -0.0002672015710464327], [0.009132595396969144, 0.04524699964717533, 0.0007423176329667283, -0.0008972487086437229, 4.210629030963312e-05,0.01994373582236024, 0.020667085763733425], [-0.00013960022773271442, 0.0005541274947928986, 7.643487654402129e-05, 4.210629030963312e-05, 0.00017645064888124678, -0.004590887129203333, 0.005255603973000529], [0.0010791432403306873, 0.021340803621331568, 0.00020524942867038474, 0.01994373582236024, -0.004590887129203333, 0.048251761048335666, 0.0005233161170690087], [-0.001029428906515696, -0.019793430266554446, -0.0002672015710464327, 0.020667085763733425, 0.005255603973000529, 0.0005233161170690087, -0.05008641718898815]], [[-0.0003977517931214619, -0.0003356177562167984, -6.520339283819167e-05, -0.00021615801381406706, -8.993674430477472e-05, -7.493449219567007e-05, 6.987536770642768e-05], [-0.0003356177562167984, 7.527074214120831e-05, -2.07504197917624e-05, 0.0007423176329667283, -7.958484674011123e-05, 0.00014384971915712617, -1.2905000916901254e-05], [-6.520339283819167e-05, -2.07504197917624e-05, 0.00029893179122863757, 0.00305980776567675, -8.472785031978486e-05, 0.0011547127645856184, -0.001121133155181657], [-0.00021615801381406706,0.0007423176329667283, 0.00305980776567675, 8.059343704781186e-05, 0.0001374402046325693, 2.52904946907122e-05, -0.00013246281919940286], [-8.993674430477472e-05, -7.958484674011123e-05, -8.472785031978486e-05, 0.0001374402046325693, -0.00023379547601553688, -0.0001372217195282382, 0.0002555986987095231], [-7.493449219567007e-05, 0.00014384971915712617, 0.0011547127645856184, 2.52904946907122e-05, -0.0001372217195282382, 5.347078666406683e-06, 5.604521858145438e-06], [6.987536770642768e-05, -1.2905000916901254e-05, -0.001121133155181657, -0.00013246281919940286, 0.0002555986987095231, 5.604521858145438e-06, 0.00032066385133268106]], [[0.27722330887011504, 0.06444595587560598, -5.122476762497382e-05, -0.0001032203144607894, 6.380603181798042e-06, 0.014596834735635524, 0.014563977287950938], [0.06444595587560598, 0.20512243441159506, -0.0004775908086346015, -0.0008972487086437229, 0.000823714107255858, 0.08269174984849849, 0.08285864175103555], [-5.122476762497382e-05, -0.0004775908086346015, 0.048850320895663844, 8.059343704781186e-05, -0.00042653821940523987, -9.229120105623288e-06, 0.00027292654501881355], [-0.0001032203144607894, -0.0008972487086437229, 8.059343704781186e-05, 0.055270848156595415, -0.00023271407765880904, 0.02593046242799854, -0.026218432398538914], [6.380603181798042e-06, 0.000823714107255858, -0.00042653821940523987, -0.00023271407765880904, 0.04979755819875569, -0.01891503044369881, -0.018648210898061998], [0.014596834735635524, 0.08269174984849849, -9.229120105623288e-06, 0.02593046242799854, -0.01891503044369881, 0.12718494331088512, 0.035028925794163106], [0.014563977287950938, 0.08285864175103555, 0.00027292654501881355, -0.026218432398538914, -0.018648210898061998, 0.035028925794163106, 0.1261122811226261]], [[6.121720965474722e-05, 0.00013188501831550432, -0.0001072088189127408, -7.444270112187904e-05, 4.056957403487189e-05, -1.648645910972081e-05, 3.198903933865263e-05], [0.00013188501831550432, 0.0003435370124715998, -8.646390311496467e-07, 4.210629030963312e-05, -0.00014870634256568182, -0.001846690953169533, 0.00197583680460194], [-0.0001072088189127408, -8.646390311496467e-07, -1.461217285854909e-05, 0.0001374402046325693, -0.00015783216485643742, 0.0002428108883273125, 0.00021019560190190965], [-7.444270112187904e-05, 4.210629030963312e-05, 0.0001374402046325693, -0.00023271407765880904, 0.0027873392162181807, -0.0013139239306251586, -0.0015806975556321861], [4.056957403487189e-05, -0.00014870634256568182, -0.00015783216485643742, 0.0027873392162181807, 0.0004298692368207001, 0.0016693285989828658, -0.0014835595477493988], [-1.648645910972081e-05, -0.001846690953169533, 0.0002428108883273125, -0.0013139239306251586, 0.0016693285989828658, -0.006879351091991742, 0.00017548664003843488], [3.198903933865263e-05, 0.00197583680460194, 0.00021019560190190965, -0.0015806975556321861,-0.0014835595477493988, 0.00017548664003843488, 0.006819246330505209]], [[0.11043082133101192, 0.026400370652775174, -2.508723781309922e-05, 0.0030109871948317563, -0.0005296233681836721, 0.006309958390147072, 0.005630234307734144], [0.026400370652775174, 0.09909683634178008, -0.0008259471732854484, 0.01994373582236024, -0.0040746475288375335, 0.05881380997639974, 0.03253185741678225], [-2.508723781309922e-05, -0.0008259471732854484, 0.02350466222068911, 2.52904946907122e-05, 0.00014527905232747124, 0.0003264507807248733, 2.976024214903177e-05], [0.0030109871948317563, 0.01994373582236024, 2.52904946907122e-05, 0.02593046242799854, -0.0013139239306251586, 0.02721434768617878, -0.003774132724812377], [-0.0005296233681836721, -0.0040746475288375335, 0.00014527905232747124, -0.0013139239306251586, 0.023918123677012657, -0.016406496158139043, -0.008916500789392826], [0.006309958390147072, 0.05881380997639974, 0.0003264507807248733, 0.02721434768617878, -0.016406496158139043, 0.12380201452690398, 0.021449859362826056], [0.005630234307734144, 0.03253185741678225, 2.976024214903177e-05, -0.003774132724812377, -0.008916500789392826, 0.021449859362826056, 0.04332644501097264]], [[-0.11206135001811636, -0.02620931048790621, 8.21415173114361e-05, 0.003123884493153085, 0.0005683521578960179, -0.005591446303681881, -0.006395962020397226], [-0.02620931048790621, -0.09827396222592277, 2.4501508025723094e-05, 0.020667085763733425, 0.004839335671706909, -0.033123083535107324,-0.05886117450806254], [8.21415173114361e-05, 2.4501508025723094e-05, -0.023457903750337957, -0.00013246281919940286, 0.00022287427477036244, 1.821817150769628e-05, -0.00024251519800809452], [0.003123884493153085, 0.020667085763733425, -0.00013246281919940286, -0.026218432398538914, -0.0015806975556321861, -0.003774132724812377, 0.026687403241767393], [0.0005683521578960179, 0.004839335671706909, 0.00022287427477036244,-0.0015806975556321861, -0.02350163680239192, 0.00875831934800763, 0.016675022890162178], [-0.005591446303681881, -0.033123083535107324, 1.821817150769628e-05, -0.003774132724812377, 0.00875831934800763, -0.04403925448238002, -0.0212366360017734], [-0.006395962020397226, -0.05886117450806254, -0.00024251519800809452, 0.026687403241767393, 0.016675022890162178, -0.0212366360017734, -0.1238312708961056]]], [[[-0.001347373877710136, 0.0001183361433619371, 4.5832447490243114e-05, -0.0001123273381929337, 0.006127301002066336, -0.00040745246894667526, -0.0003072384807615293], [0.0001183361433619371, -0.0007766239108986438, -0.00014537054807644043, -0.00013960022773271442, 0.008828512135188161, -0.0023116684027265385, -0.0020724902449363444], [4.5832447490243114e-05, -0.00014537054807644043, 1.8116830789427017e-05, -8.993674430477472e-05, 4.296156264657606e-05, 0.00013366843495048208, -5.375790698520388e-05], [-0.0001123273381929337, -0.00013960022773271442, -8.993674430477472e-05, 6.380603181798042e-06, 4.056957403487189e-05, -0.0005296233681836721,0.0005683521578960179], [0.006127301002066336, 0.008828512135188161, 4.296156264657606e-05, 4.056957403487189e-05, 5.406889972942455e-05, 0.003003656398226684, 0.0029535866730781273], [-0.00040745246894667526, -0.0023116684027265385, 0.00013366843495048208, -0.0005296233681836721, 0.003003656398226684, -0.0037830812938639533, -0.0012022442002726254], [-0.0003072384807615293, -0.0020724902449363444, -5.375790698520388e-05, 0.0005683521578960179, 0.0029535866730781273, -0.0012022442002726254, -0.0038752861527438657]], [[0.0010572226409878364, 0.0001770103305669025, 0.00024644502726346463, -3.0355259318597374e-05, 0.008828512135188161, -0.0013324615081283947, -0.0009210566467858411], [0.0001770103305669025, 0.0018136637534886739, 0.0003892658738344131, 0.0005541274947928986, 0.045920550119929245, -0.015406296197530443, -0.015883485844307852], [0.00024644502726346463, 0.0003892658738344131, -0.0004113090336712422, -7.958484674011123e-05, 7.355501211793633e-05, -0.00034317364650605133, -0.00018341887503059632], [-3.0355259318597374e-05, 0.0005541274947928986, -7.958484674011123e-05, 0.000823714107255858, -0.00014870634256568182, -0.0040746475288375335,0.004839335671706909], [0.008828512135188161, 0.045920550119929245, 7.355501211793633e-05, -0.00014870634256568182, 8.03775988054784e-05, 0.018139700242145614, 0.01765235222968264], [-0.0013324615081283947, -0.015406296197530443, -0.00034317364650605133, -0.0040746475288375335, 0.018139700242145614, -0.03879428840396135, -0.012397286852518024], [-0.0009210566467858411, -0.015883485844307852, -0.00018341887503059632, 0.004839335671706909, 0.01765235222968264, -0.012397286852518024, -0.03865119997661427]], [[0.001667507829130732, 0.00020946196158225222, 6.81974160157242e-05, 3.0189426661872054e-05, 4.296156264657606e-05, -6.900536157589576e-05, 3.965441572165417e-05], [0.00020946196158225222, -0.0007768429809752506, -8.26602344348717e-05, 7.643487654402129e-05, 7.355501211793633e-05, -4.757918058930414e-06, -9.314844761874187e-06], [6.81974160157242e-05, -8.26602344348717e-05, 0.00020712531414140026, -8.472785031978486e-05, 0.0029552031455634975, -0.0008517243170561821, -0.0009334987190590691], [3.0189426661872054e-05, 7.643487654402129e-05, -8.472785031978486e-05, -0.00042653821940523987, -0.00015783216485643742, 0.00014527905232747124, 0.00022287427477036244], [4.296156264657606e-05, 7.355501211793633e-05, 0.0029552031455634975, -0.00015783216485643742, 0.00024009755127169362, -0.0003107056100034168, -0.00016210117363589283], [-6.900536157589576e-05, -4.757918058930414e-06, -0.0008517243170561821, 0.00014527905232747124, -0.0003107056100034168, -0.00013540768263552521, -0.00012291048196992792], [3.965441572165417e-05, -9.314844761874187e-06, -0.0009334987190590691, 0.00022287427477036244, -0.00016210117363589283, -0.00012291048196992792, -0.00038059268003511786]], [[6.121720965474722e-05, 0.00013188501831550432, -0.0001072088189127408, -7.444270112187904e-05, 4.056957403487189e-05, -1.648645910972081e-05, 3.198903933865263e-05], [0.00013188501831550432, 0.0003435370124715998, -8.646390311496467e-07, 4.210629030963312e-05, -0.00014870634256568182, -0.001846690953169533, 0.00197583680460194], [-0.0001072088189127408, -8.646390311496467e-07, -1.461217285854909e-05, 0.0001374402046325693, -0.00015783216485643742, 0.0002428108883273125, 0.00021019560190190965], [-7.444270112187904e-05, 4.210629030963312e-05, 0.0001374402046325693, -0.00023271407765880904, 0.0027873392162181807, -0.0013139239306251586, -0.0015806975556321861], [4.056957403487189e-05, -0.00014870634256568182, -0.00015783216485643742, 0.0027873392162181807, 0.0004298692368207001, 0.0016693285989828658, -0.0014835595477493988], [-1.648645910972081e-05, -0.001846690953169533, 0.0002428108883273125, -0.0013139239306251586, 0.0016693285989828658, -0.006879351091991742, 0.00017548664003843488], [3.198903933865263e-05, 0.00197583680460194, 0.00021019560190190965, -0.0015806975556321861, -0.0014835595477493988, 0.00017548664003843488, 0.006819246330505209]],[[0.2473963605333805, 0.06391907629302358, -8.501073149346723e-05, 4.127458831340487e-05, 5.406889972942455e-05, 0.014734843186487162, 0.01462532540516598], [0.06391907629302358, 0.2042737745292473, 0.0005825329574675204, 0.00017645064888124678, 8.03775988054784e-05, 0.08359643466535029, 0.0813627811635329], [-8.501073149346723e-05, 0.0005825329574675204, 0.04945291619073747, -0.00023379547601553688, 0.00024009755127169362, -0.0002664188587632032, 6.163143713962752e-05], [4.127458831340487e-05, 0.00017645064888124678, -0.00023379547601553688, 0.04979755819875569, 0.0004298692368207001, 0.023918123677012657, -0.02350163680239192], [5.406889972942455e-05, 8.03775988054784e-05, 0.00024009755127169362, 0.0004298692368207001, 0.05506385583736923, -0.020132857762952993, -0.020222257395016284], [0.014734843186487162, 0.08359643466535029, -0.0002664188587632032, 0.023918123677012657, -0.020132857762952993, 0.1234343721622176, 0.0359818408741996], [0.01462532540516598, 0.0813627811635329, 6.163143713962752e-05, -0.02350163680239192, -0.020222257395016284, 0.0359818408741996, 0.12221957866780533]], [[-0.08419827364404044, -0.02038632354573043, -0.00010298315084115836, -0.0005976797023395749, 0.003003656398226684, -0.004958622289343094, -0.004900035256751265], [-0.02038632354573043, -0.07634388612024146, 0.0006080145024423654, -0.004590887129203333, 0.018139700242145614, -0.045696239623959196, -0.036367055110120235], [-0.00010298315084115836, 0.0006080145024423654, -0.018128283665638477, -0.0001372217195282382, -0.0003107056100034168, 0.00016137775509847144, 5.580975283204249e-05], [-0.0005976797023395749, -0.004590887129203333, -0.0001372217195282382, -0.01891503044369881, 0.0016693285989828658, -0.016406496158139043, 0.00875831934800763], [0.003003656398226684, 0.018139700242145614, -0.0003107056100034168, 0.0016693285989828658, -0.020132857762952993, 0.018636272354358375, 0.014401607823561823], [-0.004958622289343094, -0.045696239623959196, 0.00016137775509847144, -0.016406496158139043,0.018636272354358375, -0.0967580344376034, -0.021671574109621315], [-0.004900035256751265, -0.036367055110120235, 5.580975283204249e-05, 0.00875831934800763, 0.014401607823561823, -0.021671574109621315, -0.060920999404197776]], [[-0.08918171192738722, -0.02040994400670298, 9.348261321963548e-05, 0.0004825706122380642, 0.0029535866730781273, -0.004813244128911958, -0.004917403296289849], [-0.02040994400670298, -0.07729915678535447, 0.0004348997609271964, 0.005255603973000529, 0.01765235222968264, -0.036892983182761255, -0.045731910481121296], [9.348261321963548e-05, 0.0004348997609271964, -0.0181297242858708, 0.0002555986987095231, -0.00016210117363589283, 0.00025065663580608934, 4.959757028833822e-05], [0.0004825706122380642, 0.005255603973000529, 0.0002555986987095231, -0.018648210898061998, -0.0014835595477493988, -0.008916500789392826, 0.016675022890162178], [0.0029535866730781273, 0.01765235222968264, -0.00016210117363589283, -0.0014835595477493988, -0.020222257395016284, 0.014401607823561823, 0.018625900779955733], [-0.004813244128911958,-0.036892983182761255, 0.00025065663580608934, -0.008916500789392826, 0.014401607823561823, -0.060964080948838206, -0.021754939668849855], [-0.004917403296289849, -0.045731910481121296, 4.959757028833822e-05, 0.016675022890162178, 0.018625900779955733, -0.021754939668849855, -0.09724665596254664]]], [[[0.022267525074023773, 0.02259288695322584, 3.382843917673257e-06, 0.0005494868156957986, -0.00040745246894667526, 0.00727584391403289, 0.007096490665074045], [0.02259288695322584, 0.05855032980910715, -5.204095066859523e-05, 0.0010791432403306873, -0.0013324615081283947, 0.021706485700218457, 0.021132401282331598], [3.382843917673257e-06, -5.204095066859523e-05, 0.014696447039377591, -7.493449219567007e-05, -6.900536157589576e-05, -1.949663972696094e-05, -5.83972063184236e-05], [0.0005494868156957986, 0.0010791432403306873, -7.493449219567007e-05, 0.014596834735635524, -1.648645910972081e-05, 0.006309958390147072, -0.005591446303681881], [-0.00040745246894667526, -0.0013324615081283947, -6.900536157589576e-05, -1.648645910972081e-05, 0.014734843186487162, -0.004958622289343094, -0.004813244128911958], [0.00727584391403289, 0.021706485700218457, -1.949663972696094e-05, 0.006309958390147072, -0.004958622289343094, 0.028837548465787385, 0.008578369344337587], [0.007096490665074045, 0.021132401282331598, -5.83972063184236e-05, -0.005591446303681881, -0.004813244128911958, 0.008578369344337587, 0.02767979868837847]], [[0.3644845439441328, 0.09352806207030455, 9.173333466272427e-05, 0.003338179499542315, -0.0023116684027265385, 0.021706485700218457, 0.021266390375878876], [0.09352806207030455, 0.3287857585908965, -0.0007571370722339967, 0.021340803621331568, -0.015406296197530443, 0.1606031146011193, 0.13181022936342057], [9.173333466272427e-05, -0.0007571370722339967, 0.08081221640225682, 0.00014384971915712617, -4.757918058930414e-06, 0.00022194626511465074, -0.0002517198022295159], [0.003338179499542315, 0.021340803621331568, 0.00014384971915712617, 0.08269174984849849, -0.001846690953169533, 0.05881380997639974, -0.033123083535107324], [-0.0023116684027265385, -0.015406296197530443, -4.757918058930414e-06, -0.001846690953169533, 0.08359643466535029, -0.045696239623959196, -0.036892983182761255], [0.021706485700218457, 0.1606031146011193, 0.00022194626511465074, 0.05881380997639974, -0.045696239623959196, 0.29687599719188684, 0.06847117548415625], [0.021266390375878876, 0.13181022936342057, -0.0002517198022295159, -0.033123083535107324, -0.036892983182761255, 0.06847117548415625, 0.20165099012913748]], [[0.0005780326601597414, 0.00019677063885580035, 0.0025886606580982218, -3.973656192514144e-06, 0.00013366843495048208, -1.949663972696094e-05, 3.540002224385605e-05], [0.00019677063885580035, 0.00048227219482233183, 0.014428810501622563, 0.00020524942867038474, -0.00034317364650605133, 0.00022194626511465074, 0.0004362248572625858], [0.0025886606580982218, 0.014428810501622563, 5.4115183449543824e-05, 0.0011547127645856184, -0.0008517243170561821, 0.0055159724439013316, 0.004621082030466618], [-3.973656192514144e-06, 0.00020524942867038474, 0.0011547127645856184, -9.229120105623288e-06, 0.0002428108883273125, 0.0003264507807248733, 1.821817150769628e-05], [0.00013366843495048208, -0.00034317364650605133, -0.0008517243170561821, 0.0002428108883273125, -0.0002664188587632032, 0.00016137775509847144, 0.00025065663580608934], [-1.949663972696094e-05, 0.00022194626511465074, 0.0055159724439013316, 0.0003264507807248733, 0.00016137775509847144, 0.0002311917370747674, -6.853916750750541e-05], [3.540002224385605e-05, 0.0004362248572625858, 0.004621082030466618, 1.821817150769628e-05, 0.00025065663580608934, -6.853916750750541e-05, 0.0003106496179318075]],[[0.11043082133101192, 0.026400370652775174, -2.508723781309922e-05, 0.0030109871948317563, -0.0005296233681836721, 0.006309958390147072, 0.005630234307734144], [0.026400370652775174, 0.09909683634178008, -0.0008259471732854484, 0.01994373582236024, -0.0040746475288375335, 0.05881380997639974, 0.03253185741678225], [-2.508723781309922e-05, -0.0008259471732854484, 0.02350466222068911, 2.52904946907122e-05, 0.00014527905232747124, 0.0003264507807248733, 2.976024214903177e-05], [0.0030109871948317563, 0.01994373582236024, 2.52904946907122e-05, 0.02593046242799854, -0.0013139239306251586, 0.02721434768617878, -0.003774132724812377], [-0.0005296233681836721, -0.0040746475288375335, 0.00014527905232747124, -0.0013139239306251586, 0.023918123677012657, -0.016406496158139043, -0.008916500789392826], [0.006309958390147072, 0.05881380997639974, 0.0003264507807248733, 0.02721434768617878, -0.016406496158139043, 0.12380201452690398, 0.021449859362826056], [0.005630234307734144, 0.03253185741678225, 2.976024214903177e-05, -0.003774132724812377, -0.008916500789392826, 0.021449859362826056, 0.04332644501097264]], [[-0.08419827364404044, -0.02038632354573043, -0.00010298315084115836, -0.0005976797023395749, 0.003003656398226684, -0.004958622289343094, -0.004900035256751265], [-0.02038632354573043, -0.07634388612024146, 0.0006080145024423654, -0.004590887129203333, 0.018139700242145614, -0.045696239623959196, -0.036367055110120235], [-0.00010298315084115836, 0.0006080145024423654, -0.018128283665638477, -0.0001372217195282382, -0.0003107056100034168, 0.00016137775509847144, 5.580975283204249e-05], [-0.0005976797023395749, -0.004590887129203333, -0.0001372217195282382, -0.01891503044369881, 0.0016693285989828658, -0.016406496158139043, 0.00875831934800763], [0.003003656398226684, 0.018139700242145614, -0.0003107056100034168, 0.0016693285989828658, -0.020132857762952993, 0.018636272354358375, 0.014401607823561823], [-0.004958622289343094, -0.045696239623959196, 0.00016137775509847144, -0.016406496158139043, 0.018636272354358375, -0.0967580344376034, -0.021671574109621315], [-0.004900035256751265, -0.036367055110120235, 5.580975283204249e-05, 0.00875831934800763, 0.014401607823561823, -0.021671574109621315, -0.060920999404197776]], [[0.5149328656525697, 0.1225656953736262, -0.00029931530084244463, 0.0047679417827660945, -0.0037830812938639533, 0.028837548465787385, 0.02770084619198524], [0.1225656953736262, 0.4934603396670194, 0.001174381271699723, 0.048251761048335666, -0.03879428840396135, 0.29687599719188684, 0.2023122645187754], [-0.00029931530084244463, 0.001174381271699723, 0.117820832664402, 5.347078666406683e-06, -0.00013540768263552521, 0.0002311917370747674, -0.0005649198133347628], [0.0047679417827660945, 0.048251761048335666, 5.347078666406683e-06, 0.12718494331088512, -0.006879351091991742, 0.12380201452690398, -0.04403925448238002], [-0.0037830812938639533, -0.03879428840396135, -0.00013540768263552521, -0.006879351091991742, 0.1234343721622176, -0.0967580344376034, -0.060964080948838206], [0.028837548465787385, 0.29687599719188684, 0.0002311917370747674, 0.12380201452690398, -0.0967580344376034, 0.7461834854561834, 0.1279692584125701], [0.02770084619198524, 0.2023122645187754, -0.0005649198133347628, -0.04403925448238002, -0.060964080948838206, 0.1279692584125701, 0.3240392821500383]], [[0.15792511117601882, 0.03734740580653984, 2.418409961740715e-06, 7.258810714984401e-05, -0.0012022442002726254, 0.008578369344337587, 0.00857444586700688], [0.03734740580653984, 0.14093896484286952, -0.00022825956995994735, 0.0005233161170690087, -0.012397286852518024, 0.06847117548415625, 0.06813959308195257], [2.418409961740715e-06, -0.00022825956995994735, 0.03445784658758866, 5.604521858145438e-06, -0.00012291048196992792, -6.853916750750541e-05, 2.168793935338556e-05], [7.258810714984401e-05, 0.0005233161170690087, 5.604521858145438e-06, 0.035028925794163106, 0.00017548664003843488, 0.021449859362826056, -0.0212366360017734], [-0.0012022442002726254, -0.012397286852518024, -0.00012291048196992792, 0.00017548664003843488, 0.0359818408741996, -0.021671574109621315, -0.021754939668849855], [0.008578369344337587, 0.06847117548415625, -6.853916750750541e-05, 0.021449859362826056, -0.021671574109621315, 0.1279692584125701, 0.03477985651083896], [0.00857444586700688, 0.06813959308195257, 2.168793935338556e-05, -0.0212366360017734, -0.021754939668849855, 0.03477985651083896, 0.12742993804183303]]], [[[4.851490824336389e-05, 0.03135261126841168, -2.1658884366044888e-05, -1.5310508664913407e-05, -0.0003072384807615293, 0.007096490665074045, 0.007146804536940045], [0.03135261126841168, 0.05771947798490009, -6.474711884021497e-05, -0.001029428906515696, -0.0009210566467858411, 0.021266390375878876, 0.02184813359392102], [-2.1658884366044888e-05, -6.474711884021497e-05, 0.014563452548164666, 6.987536770642768e-05,3.965441572165417e-05, 3.540002224385605e-05, 7.883345806673966e-05], [-1.5310508664913407e-05, -0.001029428906515696, 6.987536770642768e-05, 0.014563977287950938, 3.198903933865263e-05, 0.005630234307734144, -0.006395962020397226], [-0.0003072384807615293, -0.0009210566467858411, 3.965441572165417e-05, 3.198903933865263e-05, 0.01462532540516598, -0.004900035256751265, -0.004917403296289849], [0.007096490665074045, 0.021266390375878876, 3.540002224385605e-05, 0.005630234307734144, -0.004900035256751265, 0.02770084619198524,0.00857444586700688], [0.007146804536940045, 0.02184813359392102, 7.883345806673966e-05, -0.006395962020397226, -0.004917403296289849, 0.00857444586700688, 0.028924623004966243]], [[0.4029620579613538, 0.09328429938955632, 0.00010535142003172982, -0.002731552038356108, -0.0020724902449363444, 0.021132401282331598, 0.02184813359392102], [0.09328429938955632, 0.32698320176677725, -0.0006340975237355889, -0.019793430266554446, -0.015883485844307852, 0.13181022936342057, 0.15941703232142165], [0.00010535142003172982, -0.0006340975237355889, 0.08006187313655948, -1.2905000916901254e-05, -9.314844761874187e-06, 0.0004362248572625858,0.00019962718355921692], [-0.002731552038356108, -0.019793430266554446, -1.2905000916901254e-05, 0.08285864175103555, 0.00197583680460194, 0.03253185741678225, -0.05886117450806254], [-0.0020724902449363444, -0.015883485844307852, -9.314844761874187e-06, 0.00197583680460194, 0.0813627811635329, -0.036367055110120235, -0.045731910481121296], [0.021132401282331598, 0.13181022936342057, 0.0004362248572625858, 0.03253185741678225, -0.036367055110120235, 0.2023122645187754, 0.06813959308195257], [0.02184813359392102, 0.15941703232142165, 0.00019962718355921692, -0.05886117450806254, -0.045731910481121296, 0.06813959308195257, 0.2993652344262104]], [[0.0006112624549165606, -1.8227093133976622e-05, 0.0024911255398025658, 2.4377154310510185e-05, -5.375790698520388e-05, -5.83972063184236e-05, 7.883345806673966e-05], [-1.8227093133976622e-05, -0.0002905224191685269, 0.014867452577030362, -0.0002672015710464327, -0.00018341887503059632, -0.0002517198022295159, 0.00019962718355921692], [0.0024911255398025658, 0.014867452577030362, 0.00014710386548829533, -0.001121133155181657, -0.0009334987190590691, 0.004621082030466618, 0.005702058346928917], [2.4377154310510185e-05, -0.0002672015710464327, -0.001121133155181657, 0.00027292654501881355, 0.00021019560190190965, 2.976024214903177e-05, -0.00024251519800809452], [-5.375790698520388e-05, -0.00018341887503059632, -0.0009334987190590691, 0.00021019560190190965, 6.163143713962752e-05, 5.580975283204249e-05, 4.959757028833822e-05], [-5.83972063184236e-05, -0.0002517198022295159, 0.004621082030466618, 2.976024214903177e-05, 5.580975283204249e-05, -0.0005649198133347628, 2.168793935338556e-05], [7.883345806673966e-05, 0.00019962718355921692, 0.005702058346928917, -0.00024251519800809452, 4.959757028833822e-05, 2.168793935338556e-05, 0.00046133282878836726]], [[-0.11206135001811636, -0.02620931048790621, 8.21415173114361e-05, 0.003123884493153085, 0.0005683521578960179, -0.005591446303681881, -0.006395962020397226], [-0.02620931048790621, -0.09827396222592277, 2.4501508025723094e-05, 0.020667085763733425, 0.004839335671706909, -0.033123083535107324, -0.05886117450806254], [8.21415173114361e-05, 2.4501508025723094e-05, -0.023457903750337957, -0.00013246281919940286, 0.00022287427477036244, 1.821817150769628e-05, -0.00024251519800809452], [0.003123884493153085, 0.020667085763733425, -0.00013246281919940286, -0.026218432398538914, -0.0015806975556321861, -0.003774132724812377, 0.026687403241767393], [0.0005683521578960179, 0.004839335671706909, 0.00022287427477036244, -0.0015806975556321861, -0.02350163680239192, 0.00875831934800763, 0.016675022890162178], [-0.005591446303681881, -0.033123083535107324, 1.821817150769628e-05, -0.003774132724812377, 0.00875831934800763, -0.04403925448238002, -0.0212366360017734], [-0.006395962020397226, -0.05886117450806254, -0.00024251519800809452, 0.026687403241767393, 0.016675022890162178, -0.0212366360017734, -0.1238312708961056]], [[-0.08918171192738722, -0.02040994400670298, 9.348261321963548e-05, 0.0004825706122380642, 0.0029535866730781273, -0.004813244128911958, -0.004917403296289849], [-0.02040994400670298, -0.07729915678535447, 0.0004348997609271964, 0.005255603973000529, 0.01765235222968264, -0.036892983182761255, -0.045731910481121296], [9.348261321963548e-05, 0.0004348997609271964, -0.0181297242858708, 0.0002555986987095231, -0.00016210117363589283, 0.00025065663580608934, 4.959757028833822e-05], [0.0004825706122380642, 0.005255603973000529, 0.0002555986987095231, -0.018648210898061998, -0.0014835595477493988, -0.008916500789392826, 0.016675022890162178], [0.0029535866730781273, 0.01765235222968264, -0.00016210117363589283, -0.0014835595477493988, -0.020222257395016284, 0.014401607823561823, 0.018625900779955733], [-0.004813244128911958, -0.036892983182761255, 0.00025065663580608934, -0.008916500789392826, 0.014401607823561823, -0.060964080948838206, -0.021754939668849855], [-0.004917403296289849, -0.045731910481121296, 4.959757028833822e-05, 0.016675022890162178, 0.018625900779955733, -0.021754939668849855, -0.09724665596254664]], [[0.15792511117601882, 0.03734740580653984, 2.418409961740715e-06, 7.258810714984401e-05, -0.0012022442002726254, 0.008578369344337587, 0.00857444586700688], [0.03734740580653984, 0.14093896484286952, -0.00022825956995994735, 0.0005233161170690087, -0.012397286852518024, 0.06847117548415625, 0.06813959308195257], [2.418409961740715e-06, -0.00022825956995994735, 0.03445784658758866, 5.604521858145438e-06, -0.00012291048196992792, -6.853916750750541e-05, 2.168793935338556e-05], [7.258810714984401e-05, 0.0005233161170690087, 5.604521858145438e-06, 0.035028925794163106, 0.00017548664003843488, 0.021449859362826056, -0.0212366360017734], [-0.0012022442002726254, -0.012397286852518024, -0.00012291048196992792, 0.00017548664003843488, 0.0359818408741996, -0.021671574109621315, -0.021754939668849855], [0.008578369344337587, 0.06847117548415625, -6.853916750750541e-05, 0.021449859362826056, -0.021671574109621315, 0.1279692584125701, 0.03477985651083896], [0.00857444586700688, 0.06813959308195257, 2.168793935338556e-05, -0.0212366360017734, -0.021754939668849855, 0.03477985651083896, 0.12742993804183303]], [[0.5170677309939025, 0.12283131547133579, 2.8845205573384447e-06, -0.004884236643421517, -0.0038752861527438657, 0.02767979868837847, 0.028924623004966243], [0.12283131547133579, 0.49648660188975735, 0.0007511070389618449, -0.05008641718898815, -0.03865119997661427, 0.20165099012913748, 0.2993652344262104], [2.8845205573384447e-06, 0.0007511070389618449, 0.11742614051785429, 0.00032066385133268106, -0.00038059268003511786, 0.0003106496179318075, 0.00046133282878836726], [-0.004884236643421517, -0.05008641718898815, 0.00032066385133268106, 0.1261122811226261, 0.006819246330505209, 0.04332644501097264, -0.1238312708961056], [-0.0038752861527438657, -0.03865119997661427, -0.00038059268003511786, 0.006819246330505209, 0.12221957866780533, -0.060920999404197776, -0.09724665596254664], [0.02767979868837847, 0.20165099012913748, 0.0003106496179318075, 0.04332644501097264, -0.060920999404197776, 0.3240392821500383, 0.12742993804183303], [0.028924623004966243, 0.2993652344262104, 0.00046133282878836726, -0.1238312708961056, -0.09724665596254664, 0.12742993804183303, 0.7375028699018835]]]]

eri = np.array(eri)
print(eri.shape)

(7, 7, 7, 7)


#6. Solve F'C' = C'e for C' and e, then transform C' -> C -> P. Re-formulate F (the J and K integrals) using P, solve for C' and e again and repeat until e converges. Also add nuclear repulsion energy to E_0.

In [143]:
# Calculate nuclear repulsion energy

E_nuc = 0.0
for i in range(len(Molecule["Atom_label"])):
    for j in range(len(Molecule["Atom_label"])):
        if i == j:
            continue  # Exclude i = j terms
        Z1, Z2 = Molecule['Z'][i], Molecule['Z'][j]
        R = np.sqrt((Molecule['x'][i] - Molecule['x'][j])**2 +
                    (Molecule['y'][i] - Molecule['y'][j])**2 +
                    (Molecule['z'][i] - Molecule['z'][j])**2)
        E_nuc += 0.5 * Z1 * Z2 / R
print("Nuclear Repulsion Energy =", E_nuc, "Hartrees")

# N.B. Factor of 0.5 to avoid double counting

# Find J, K and F, and Transform F -> F' = X^T F X

def J_and_K_finder(P, eri, basis_set):
    J = np.zeros((len(basis_set), len(basis_set)))
    K = np.zeros((len(basis_set), len(basis_set)))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    J[i, j] += P[k, l] * eri[i, j, k, l]
                    K[i, j] += P[k, l] * eri[i, k, j, l]
    return J, K

J, K = J_and_K_finder(P, eri, basis_set)


E_nuc = 0

def compute_E_HF(T, V, J, K, E_nuc):
    H_core = T + V
    E_one_electron = 0
    E_two_electron = 0
    for i in range(n_occ):
      E_one_electron += 2 * H_core[i, i]
      for j in range(n_occ):
        E_two_electron += 2 * J[i, j] - K[i, j]
    E_total = E_one_electron + E_two_electron + E_nuc
    return E_total

# Provide initial guess for F
J, K = J_and_K_finder(P, eri, basis_set)
F = T + V + J - 0.5*K

def SCF_iterator(F, X, T, V, basis_set, n_occ, E_nuc, eri):

  F_prime = X.T @ F @ X

  # Obtain eigenvectors (C') of the problem: (F' - e)(C') = 0
  eigenenergies, C_prime = np.linalg.eig(F_prime)
  # Order C_prime columns in order of the eigenenergies
  idx = eigenenergies.argsort()
  C_prime = C_prime[:, idx]

  # Convert C' into C via # C = X C'
  X_inv = np.linalg.inv(X)
  C = X @ C_prime
  C = np.array(C)
  C_occ = C[:, :n_occ]

  # Find new P matrix
  P = np.zeros((len(basis_set), len(basis_set)))
  for i in range(n_occ):
    for mu in range(len(basis_set)):
      for nu in range(len(basis_set)):
        P[mu, nu] += 2*(C_occ[mu, i] * C_occ[nu, i])

  # Call new J and K functions (T, V, S, X do not change throughout SCF iterations)
  J, K = J_and_K_finder(P, eri, basis_set)

  F = np.zeros((len(basis_set), len(basis_set)))
  for i in range(len(basis_set)):
    for j in range(len(basis_set)):
      F[i, j] = T[i, j] + V[i, j] + J[i, j] - 0.5 * K[i, j]

  E_HF = compute_E_HF(T, V, J, K, E_nuc)

  return P, F, E_HF


for i in range(20):
  P, F, E_HF = SCF_iterator(F, X, T, V, basis_set, n_occ, E_nuc, eri)
  print("E_HF =", E_HF, "Hartrees")

Nuclear Repulsion Energy = 9.189361438718237 Hartrees
E_HF = -74.45271587204577 Hartrees
E_HF = -74.33985026825496 Hartrees
E_HF = -74.3359119090981 Hartrees
E_HF = -74.33575524408762 Hartrees
E_HF = -74.33574878199295 Hartrees
E_HF = -74.3357485068793 Hartrees
E_HF = -74.33574849482147 Hartrees
E_HF = -74.33574849427907 Hartrees
E_HF = -74.33574849425412 Hartrees
E_HF = -74.33574849425295 Hartrees
E_HF = -74.33574849425287 Hartrees
E_HF = -74.33574849425287 Hartrees
E_HF = -74.33574849425287 Hartrees
E_HF = -74.33574849425287 Hartrees
E_HF = -74.33574849425287 Hartrees
E_HF = -74.33574849425285 Hartrees
E_HF = -74.33574849425287 Hartrees
E_HF = -74.33574849425287 Hartrees
E_HF = -74.33574849425285 Hartrees
E_HF = -74.33574849425287 Hartrees
